    Position                       Name + ID                 Name        ID  \
0         WR     Justin Jefferson (28792603)     Justin Jefferson  28792603   
1         RB  Christian McCaffrey (28792269)  Christian McCaffrey  28792269   
2         RB        Austin Ekeler (28792271)        Austin Ekeler  28792271   
3         WR          Cooper Kupp (28792605)          Cooper Kupp  28792605   
4         RB           Nick Chubb (28792273)           Nick Chubb  28792273   
..       ...                             ...                  ...       ...   
820       TE      Camren McDonald (28793407)      Camren McDonald  28793407   
821      DST               Colts  (28793597)               Colts   28793597   
822      DST             Raiders  (28793598)             Raiders   28793598   
823      DST          Buccaneers  (28793599)          Buccaneers   28793599   
824      DST              Texans  (28793600)              Texans   28793600   

    Roster Position  Salary                      Ga

In [113]:
import time
import pandas as pd
import numpy as np
import re as re
from pulp import *


problem = LpProblem('maximize', LpMaximize)
sum_lineup_players=9
salary_cap = 50000
players = pd.read_excel('Week1_Proj.xlsx')
availables = players.groupby(['Position','ID','Name + ID','Proj','Salary']).agg('count').reset_index()
availables.to_csv('availables.csv')
salaries = {}
points = {}
all_pos = {'QB': 1, 'RB': 2, 'WR': 3, 'TE': 1, 'FLEX': 1, 'DST': 1}
flex_pos = ['RB', 'WR', 'TE']

#position constraints
pos_num_fixed = {k:v for k, v in all_pos.items() if (k not in flex_pos and k!='FLEX')}
if 'FLEX' in all_pos.values():
    pos_num_less_equal = {k:v+all_pos['FLEX'] for k, v in all_pos.items() if (k in flex_pos and k!='FLEX')}
    pos_num_greater_equal = {k:v for k, v in all_pos.items() if (k in flex_pos and k!='FLEX')}

#Create a dictionaries by position for both salary and projection
for pos in availables.Position.unique():
    available_pos = availables[availables.Position==pos]
    salary = list(available_pos[['ID', 'Salary']].set_index('ID').to_dict().values())[0]
    point = list(available_pos[['ID', 'Proj']].set_index('ID').to_dict().values())[0]
    salaries[pos] = salary
    points[pos] = point

_vars = {k: LpVariable.dict(k, v, cat='Binary') for k, v in points.items()}#Changed v from a proj to an LpVariable
rewards=[]
costs=[]

#Add positional constraints and create
for k, v in _vars.items():
    costs += lpSum([salaries[k][i] * _vars[k][i] for i in v])
    rewards += lpSum([points[k][i] * _vars[k][i] for i in v])
    if k in pos_num_fixed:
        problem += lpSum([_vars[k][i] for i in v]) == pos_num_fixed[k]
    if k in pos_num_less_equal:
        problem += lpSum([_vars[k][i] for i in v]) <= pos_num_less_equal[k]
    if k in pos_num_greater_equal:
        problem += lpSum([_vars[k][i] for i in v]) >= pos_num_greater_equal[k]


total_players_constraint = lpSum([v.values() for v in _vars.values()]) == sum_lineup_players
problem += total_players_constraint
salary_cap_constraint = lpSum(costs) <= salary_cap
problem += salary_cap_constraint
objective = lpSum(rewards)#no constraint so will be treated like the objective 
problem += objective
tic=time.perf_counter()
problem.solve()
toc=time.perf_counter()
print('time elapsed: '+str(toc-tic))


#print(problem)




Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/matthewmoore/.local/share/virtualenvs/optimizer_v1-vT5i3nNP/lib/python3.11/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/1q/3wk2pn_x6rdgn29l0fs4j47r0000gn/T/5eb5854762e64ac3b3ae032f6e174140-pulp.mps max branch printingOptions all solution /var/folders/1q/3wk2pn_x6rdgn29l0fs4j47r0000gn/T/5eb5854762e64ac3b3ae032f6e174140-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 5072 RHS
At line 5083 BOUNDS
At line 5909 ENDATA
Problem MODEL has 10 rows, 825 columns and 3178 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 142.384 - 0.01 seconds
Cgl0003I 38 fixed, 0 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 38 fixed, 0 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 7 rows, 223 columns (223 integer (199 of which binary)) and 669 elements
Cutoff increme

In [105]:
dir(problem)
print(list(map(LpVariable.toDict,problem.variables())))
print(problem.variables())

LpVariable.toDict

{'hi': 'hi'}.pop('flex')

      




[{'lowBound': 0, 'upBound': 1, 'cat': 'Integer', 'varValue': 0.0, 'dj': 9.0, 'name': 'DST_28793575'}, {'lowBound': 0, 'upBound': 1, 'cat': 'Integer', 'varValue': 0.0, 'dj': 7.3, 'name': 'DST_28793576'}, {'lowBound': 0, 'upBound': 1, 'cat': 'Integer', 'varValue': 0.0, 'dj': 7.2, 'name': 'DST_28793577'}, {'lowBound': 0, 'upBound': 1, 'cat': 'Integer', 'varValue': 0.0, 'dj': 8.9, 'name': 'DST_28793578'}, {'lowBound': 0, 'upBound': 1, 'cat': 'Integer', 'varValue': 0.0, 'dj': 6.6, 'name': 'DST_28793579'}, {'lowBound': 0, 'upBound': 1, 'cat': 'Integer', 'varValue': 0.0, 'dj': 7.9, 'name': 'DST_28793580'}, {'lowBound': 0, 'upBound': 1, 'cat': 'Integer', 'varValue': 0.0, 'dj': 8.2, 'name': 'DST_28793581'}, {'lowBound': 0, 'upBound': 1, 'cat': 'Integer', 'varValue': 0.0, 'dj': 8.8, 'name': 'DST_28793582'}, {'lowBound': 0, 'upBound': 1, 'cat': 'Integer', 'varValue': 0.0, 'dj': 7.4, 'name': 'DST_28793583'}, {'lowBound': 0, 'upBound': 1, 'cat': 'Integer', 'varValue': 0.0, 'dj': 6.6, 'name': 'DST_2

KeyError: 'flex'

classmethoddicts(name, indices=None, lowBound=None, upBound=None, cat='Continuous', indexStart=[], indexs=None)
This function creates a dictionary of LpVariable with the specified associated parameters.
Parameters:
name – The prefix to the name of each LP variable created
indices – A list of strings of the keys to the dictionary of LP variables, and the main part of the variable name itself
lowBound – The lower bound on these variables’ range. Default is negative infinity
upBound – The upper bound on these variables’ range. Default is positive infinity
cat – The category these variables are in, Integer or Continuous(default)
indexs – (deprecated) Replaced with indices parameter
Returns:
A dictionary of LpVariable

In [60]:



def optimizer():
    
    lineups='hi'

    return lineups

optimizer()

class Slate():
    def __init__(self, players=players):
        pass

    def calc_optimal_lineup(self):
        pass


class Lineup():
    pass


class Player():
    pass




'hi'

In [81]:
class Player():
    def __init__(self, first_name, last_name):
        self.first_name=first_name
        self.last_name=last_name


for byte in b'Binary':
    print(byte)



test_dict={
    'k1': 'hello',
    'k2': 'hi'
}

'k1' in test_dict

for k, v in _vars.items():
    #print(v.values())
    pass


list1 = [1, 2, 3]
list2 = ['a', 'b']

combinations = [(x, y) for x in list1 for y in list2]

print(combinations)
#[(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b'), (3, 'a'), (3, 'b')]



#for k, v in _vars.items():
#print([nested_item for v in nested_dict.values() for nested_dict in _vars])

for v in _vars.values():
    print(v.values())


print(total_players_rule)








    




66
105
110
97
114
121
[(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b'), (3, 'a'), (3, 'b')]
dict_values([DST_28793575, DST_28793576, DST_28793577, DST_28793578, DST_28793579, DST_28793580, DST_28793581, DST_28793582, DST_28793583, DST_28793584, DST_28793585, DST_28793586, DST_28793587, DST_28793588, DST_28793589, DST_28793590, DST_28793591, DST_28793592, DST_28793593, DST_28793594, DST_28793595, DST_28793596, DST_28793597, DST_28793598, DST_28793599, DST_28793600])
dict_values([QB_28792178, QB_28792179, QB_28792180, QB_28792181, QB_28792182, QB_28792183, QB_28792184, QB_28792185, QB_28792186, QB_28792187, QB_28792188, QB_28792189, QB_28792190, QB_28792191, QB_28792192, QB_28792193, QB_28792194, QB_28792195, QB_28792196, QB_28792197, QB_28792198, QB_28792199, QB_28792200, QB_28792201, QB_28792202, QB_28792203, QB_28792204, QB_28792205, QB_28792206, QB_28792207, QB_28792208, QB_28792209, QB_28792210, QB_28792211, QB_28792212, QB_28792213, QB_28792214, QB_28792215, QB_28792216, QB_28792217, QB_287